# Data Mining of Human Skin Microbiome from EBI-Metagenomics Portal

_Matin Nuhamunada_<sup>1*</sup>, _Gregorius Altius Pratama_<sup>1</sup>, _Setianing Wikanthi_<sup>2</sup>, and _Mohamad Khoirul Anam_<sup>1</sup>

<sup>1</sup>Department of Tropical Biology, Universitas Gadjah Mada;   
Jl. Teknika Selatan, Sekip Utara, Bulaksumur, Yogyakarta, Indonesia, 55281;   

<sup>2</sup>Department of Agricultural Microbiology, Universitas Gadjah Mada;  

*Correspondence: [matin_nuhamunada@ugm.ac.id](mailto:matin_nuhamunada@mail.ugm.ac.id)  
[mohamad.khoirul.anam@mail.ugm.ac.id](mailto:mohamad.khoirul.anam@mail.ugm.ac.id)  
[gregorius.altius.p@mail.ugm.ac.id](mailto:gregorius.altius.p@mail.ugm.ac.id)  
[setianingwikanthi@mail.ugm.ac.id](mailto:setianingwikanthi@mail.ugm.ac.id)

---
## Abstract
Human skin microbiome is unique to individuals in regards to many aspects, including behaviour, environment, and perhaps maybe genes. To understand more about the distribution of human skin microbiome across the globe, we compare several skin microbiome study available in the EBI-Metagenomic Portal. Study data was acquired using EBI-Metagenome API, and sample data was selected based on sex, location, and bodysite. The biological observation matrix from the analysis result of the selected samples were compared using MEGAN. 

### Keywords
Human Skin, Microbiome, EBI-Metagenome


## Import Python Modules
We use python 3 script using ``pandas``, ``jsonapi_client``, ``pycurl``, to mine the data from EBI-metagenomic portal  [[1]](#ref1).

In [4]:
from pandas import DataFrame
import pandas as pd

try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

In [5]:
from jsonapi_client import Session, Filter

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [6]:
import ipywidgets
import pycurl
import os, sys
from tqdm import tqdm_notebook
import numpy as np
from time import sleep
import glob

## Load Functions

In [7]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

def get_study(term, lineage, biome, filename):
    if not os.path.isfile(filename):
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/projects/doExportDetails?searchTerm='+term+'&includingChildren=true&biomeLineage=root%3A'+lineage+'%3A'+biome+'&search=Search')
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    return filename

def get_analysis_result(run, extension):
    API_BASE_RUN = 'https://www.ebi.ac.uk/metagenomics/api/latest/runs'
    with Session(API_BASE_RUN) as s:
        study = s.get(run,'analysis').resource
        for i in study.downloads:
            if extension in i.file_format['name']:
                link = i.url
    return link

## Get Study
We search the EBI Metagenomic database on human skin microbiome study in the host-associated biome with 'skin' as search term. The study list can be found on this link: https://www.ebi.ac.uk/metagenomics/projects/doExportDetails?searchTerm=skin&includingChildren=true&biomeLineage=root%3AHost-associated%3AHuman&search=Search

In [8]:
#Search Study
term = 'skin'
lineage = 'Host-associated'
biome = 'Human'
file_study = '01_Study_'+term+'+'+biome+'+'+lineage+'_raw.csv'

In [9]:
#Download study information
get_study(term, biome, lineage, file_study)

'01_Study_skin+Human+Host-associated_raw.csv'

In [10]:
#Load Study information
df1 = pd.read_csv(file_study)
#df1

In [11]:
#Select relevant information
df_study = df1[["Study ID","Study Name","Number Of Samples", "Submitted Date", "Experimental Factor", "Study Abstract"]]
#df_study

In [12]:
#Add information on biome
df2 = DataFrame(columns=("Biome","Lineage","Publication"))
#df2.index.name = 'No'

for i in tqdm_notebook(range(len(df_study))):
    with Session(API_BASE) as s:
        std = s.get('studies', df_study.loc[i, "Study ID"]).resource
        for a in std.biomes:
            df2.loc[i] = [a.biome_name,
                          a.lineage,
                          std.publications
        ]
#df2

In [13]:
#Merge & Filter Table
df3 = pd.concat([df_study, df2], axis=1)
df3.set_index(["Study ID"])
df_study_biome = df3.query('Biome == ["Human", "Skin"] and Lineage == ["root:Host-associated:Human:Skin"]')
df_study_biome

,Study ID,Study Name,Number Of Samples,Submitted Date,Experimental Factor,Study Abstract,Biome,Lineage,Publication
1,SRP002480,Gene-Environment Interactions at the Skin Surface,2560,2016-02-03,NaN,16S rRNA gene sequences amplified from subject...,Skin,root:Host-associated:Human:Skin,[publications: 22310478 (2134965149312)]
2,ERP018577,Human skin bacterial and fungal microbiotas an...,96,2016-11-03,NaN,Using high-throughput 16S rDNA and ITS1 sequen...,Skin,root:Host-associated:Human:Skin,[]
3,ERP022958,Impact of the Mk VI SkinSuit on skin microbiot...,204,2017-06-16,NaN,Microgravity induces physiological decondition...,Skin,root:Host-associated:Human:Skin,[]
4,ERP019566,Longitudinal study of the diabetic skin and wo...,258,2017-11-27,NaN,Background: Type II diabetes is a chronic heal...,Skin,root:Host-associated:Human:Skin,[publications: 28740749 (2134965232808)]
5,ERP016629,Microbiome samples derived from Buruli ulcer w...,14,2016-07-29,NaN,Background: Buruli ulcer (BU) is an infectious...,Skin,root:Host-associated:Human:Skin,[publications: 28750103 (2134965128664)]
7,SRP056364,Skin microbiome in human volunteers inoculated...,191,2016-02-04,Infection,The aim of this project was to investigate the...,Skin,root:Host-associated:Human:Skin,[]


In [14]:
#Export study data
df_study_biome.to_csv('02_Study_'+term+'+'+biome+'+'+lineage+'_filtered.csv')

### Choose relevant study
Our search using the keyterm "skin" on Human host-associated biome found 7 hits of studies. Of this studies, we tried to select comparable sample of human skin microbiome to get more understanding of human microbiome profile from different studies.

From the data above, we can filter which study can be used as the data for comparative microbiome analysis of human skin samples. Therefore we choose the study ID 'SRP002480' as our data

In [15]:
#Selected study
study = 'SRP002480'

### Get list of sample for a given study and its metadata

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480
List samples: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480/samples
Fetch samples for the given study accession: https://www.ebi.ac.uk/metagenomics/api/latest/samples?study_accession=SRP002480


In [16]:
#Fetch a list of sample data from a given study
filename_sample = '03_sample_'+study+'_raw.csv'
print(filename_sample)
if not os.path.isfile(filename_sample):
    with open(filename_sample, 'wb') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, 'https://www.ebi.ac.uk/metagenomics/projects/'+study+'/overview/doExport')
        c.setopt(c.WRITEDATA, f)
        c.perform()
        c.close()

03_sample_SRP002480_raw.csv


In [17]:
#Filter relevant information from the list
df_sample = pd.read_csv(filename_sample)
df_sample_refine = df_sample[["Sample ID","Run ID","Release version"]]

In [18]:
#Create Container
if not os.path.isfile('04_raw_meta_'+study+'.csv'):
    df_meta = DataFrame(columns=('Sex',"Body site"))
    df_meta.index.name = 'No'
else:
    df_meta = pd.read_csv('04_raw_meta_'+study+'.csv', index_col=0)

#Fetch metadata for given sample
pbar = tqdm_notebook(range(len(df_sample_refine))) #to make progressbar
for i in pbar:
    if not i in df_meta.index:
        with Session(API_BASE) as s:
            s_meta = s.get('samples', df_sample_refine.loc[i, "Sample ID"]).resource
            df_meta.loc[i] = [
                get_metadata(s_meta.sample_metadata, 'sex'),
                get_metadata(s_meta.sample_metadata, 'body site')
            ]
        pbar.set_description('processed: %d' % (i))
        pbar.update(1)
        sleep(1)

In [19]:
#Write to container
df_meta.to_csv('04_raw_meta_'+study+'.csv')
#df_meta

In [20]:
#Merge metadata with the raw sample list
result = pd.concat([df_sample_refine, df_meta], axis=1)
result.to_csv('05_sample_'+study+'_meta.csv')

### Query samples based on its metadata

In [21]:
#Load sample list
df_result_raw = pd.read_csv('05_sample_'+study+'_meta.csv', index_col = 0)

In [22]:
#Refine the column name to make it easier for filtering 
df_result_raw.columns = df_result_raw.columns.str.replace(' ', '_') 
#df_result_raw

In [23]:
#Delete empty datas
df = df_result_raw
df_result_refine = df[df_result_raw != '#N/A ']
df_result_refine = df_result_refine.dropna()
df_result_refine.to_csv('05_sample_'+study+'_meta_refine.csv')
df_result_refine

,Sample_ID,Run_ID,Release_version,Sex,Body_site
0,SRS451417,SRR919527,2.0,male,antecubital crease
1,SRS451417,SRR919587,2.0,male,antecubital crease
2,SRS451418,SRR919528,2.0,male,back
3,SRS451418,SRR919588,2.0,male,back
4,SRS451419,SRR919529,2.0,male,external auditory canal
5,SRS451419,SRR919589,2.0,male,external auditory canal
6,SRS451420,SRR919530,2.0,male,hypothenar palm
7,SRS451420,SRR919590,2.0,male,hypothenar palm
8,SRS451421,SRR919531,2.0,male,retroauricular crease
9,SRS451421,SRR919591,2.0,male,retroauricular crease


In [36]:
#Create a list of values in body site metadata
meta_list = df_result_refine["Body_site"].tolist()
meta_list_refine = list(set(meta_list))
meta_list_refine

['toeweb ',
 'plantar heel ',
 'nare ',
 'volar forearm ',
 'popliteal fossa ',
 'manubrium ',
 'antecubital fossa ',
 'external auditory canal ',
 'inguinal crease ',
 'glabella ',
 'occiput ',
 'toenail ',
 'retroauricular crease ',
 'hypothenar palm ',
 'back ',
 'antecubital crease ',
 'vagina ']

In [37]:
#Create dataframes based on sorted metadata
def sort_by_meta(dataframe, sex, body_site): 
        return df.query('Sex==@sex and Body_site==@body_site')

df = df_result_refine
sex_cat = ['female ', 'male ']
meta_ls = meta_list_refine[:-1]
df_merge = pd.DataFrame(columns=('Sample_ID', 'Run_ID', 'Release_version', 'Sex', 'Body_site'))
for i in range(len(meta_ls)):
    bs = meta_ls[i]
    for a in range(len(sex_cat)):
        df_sort = sort_by_meta(df, sex_cat[a], bs)
        dataframe = df_sort
        amount = 5
        df_random_sample = random_sampling(dataframe, amount)
        df_merge = pd.concat([df_merge, df_random_sample], ignore_index=True)
df_merge

,Sample_ID,Run_ID,Release_version,Sex,Body_site
0,SRS452123,SRR921059,2.0,female,toeweb
1,SRS451751,SRR920153,2.0,female,toeweb
2,SRS451761,SRR920241,2.0,female,toeweb
3,SRS451782,SRR920231,2.0,female,toeweb
4,SRS451473,SRR919583,2.0,female,toeweb
5,SRS452059,SRR920933,2.0,male,toeweb
6,SRS451727,SRR920129,2.0,male,toeweb
7,SRS451935,SRR920645,2.0,male,toeweb
8,SRS451454,SRR919564,2.0,male,toeweb
9,SRS451618,SRR919889,2.0,male,toeweb


In [42]:
df_merge.to_csv('data_all.csv')

#### Automatize the body site metadata input
for i in range(len(meta_list_refine)): 
    bs = meta_list_refine[i]  
    #Create sorted dataframes
    sort_by_meta(df, s, bs)

#### Create .csv files of sorted dataframes
    filename_sample = 'sort_by_meta ' + s + bs + '.csv' 
    print(filename_sample) #To check the files' names
    if not os.path.isfile(filename_sample):
        sort_by_meta(df, s, bs).to_csv('sort_by_meta ' + s + bs + '.csv')

In [153]:
cwd = os.getcwd() #get current working directory
os.chdir('D:\Jupyter\KetiakProject\src\sorted files')

filenames = glob.glob('sort_by_meta*.csv')

count_list = []

for i in range(len(filenames)):
    print(filenames[i])
    n = sum(1 for line in open(filenames[i])) - 1
    print(n)
    count_list.append(n)

minimum_rows = min(count_list)
maximum_rows = max(count_list)
print('Minimum rows:', minimum_rows)    
print('Maximum rows:', maximum_rows)

os.chdir(cwd)

sort_by_meta female antecubital crease .csv
20
sort_by_meta female antecubital fossa .csv
274
sort_by_meta female back .csv
10
sort_by_meta female external auditory canal .csv
20
sort_by_meta female glabella .csv
10
sort_by_meta female hypothenar palm .csv
22
sort_by_meta female inguinal crease .csv
20
sort_by_meta female manubrium .csv
12
sort_by_meta female nare .csv
168
sort_by_meta female occiput .csv
14
sort_by_meta female plantar heel .csv
28
sort_by_meta female popliteal fossa .csv
84
sort_by_meta female retroauricular crease .csv
112
sort_by_meta female toenail .csv
28
sort_by_meta female toeweb .csv
26
sort_by_meta female vagina .csv
2
sort_by_meta female volar forearm .csv
214
sort_by_meta male antecubital crease .csv
44
sort_by_meta male antecubital fossa .csv
480
sort_by_meta male back .csv
22
sort_by_meta male external auditory canal .csv
44
sort_by_meta male glabella .csv
46
sort_by_meta male hypothenar palm .csv
44
sort_by_meta male inguinal crease .csv
44
sort_by_meta m

## Random sampling

In [25]:
def random_sampling(dataframe, amount):
    df_random = DataFrame(columns=('Sample_ID','Run_ID','Release_version','Sex','Body_site'))
    df_random.index.name = 'No'
    a = 0
    while a < amount:
        i = np.random.choice(dataframe.index.values, 1)
        container = df_random.loc[:, 'Sample_ID']
        if not container.isin([dataframe.loc[i[0], 'Sample_ID']]).any():
            df_random.loc[i[0]] = [dataframe.loc[i[0], 'Sample_ID'], \
                                   dataframe.loc[i[0], 'Run_ID'], \
                                   dataframe.loc[i[0], 'Release_version'], \
                                   dataframe.loc[i[0], 'Sex'], \
                                   dataframe.loc[i[0], 'Body_site']
                                  ]
            a = a + 1
    return df_random

In [40]:
dataframe = df_sort
amount = 5
df_random_sample = random_sampling(dataframe, amount)
df_random_sample

,Sample_ID,Run_ID,Release_version,Sex,Body_site
No,,,,,
60,SRS451447,SRR919557,2.0,male,occiput
606,SRS452054,SRR920927,2.0,male,occiput
655,SRS452078,SRR921014,2.0,male,occiput
249,SRS451723,SRR920172,2.0,male,occiput
779,SRS452140,SRR921140,2.0,male,occiput


In [22]:
df_random_sample.to_csv('06_sampled_biom_'+study+'.csv')

### Get analysis result of a given sample in a study
https://www.ebi.ac.uk/metagenomics//projects/SRP002480/samples/SRS451457/runs/SRR919567/results/versions/2.0/taxonomy/OTU-table-HDF5-BIOM

In [39]:
#Create ouput folder
cwd = os.getcwd() #get current working directory
output_folder = "\output_"+study #name of the output folder for a given study, use \ for directory in windows
if not os.path.isdir(cwd + output_folder):
    os.mkdir(cwd + output_folder)
new_dir = cwd + output_folder 
new_dir

'E:\\Jupyter_Lab\\KetiakProject\\src\\output_SRP002480'

In [44]:
extension = 'JSON Biom'
df_result_all = pd.read_csv('data_all.csv')

#Ambil data dari EBI
for i in tqdm_notebook(df_result_all.index):
    os.chdir(new_dir) #pindah ke folder output
    filename = df_result_all.loc[i, "Sample_ID"]+'.biom'
    if not os.path.isfile(filename):
        link = get_analysis_result(df_result_all.loc[i, "Run_ID"], extension)
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, link)
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    os.chdir(cwd) #balik ke folder semula

## Download accompanying metadata for downloaded file

In [46]:
df_meta_random = DataFrame(columns=('#SampleID','BarcodeSequence','LinkerPrimerSequence', 'RunID', 'Sex', 'BodySite', 'Description'))

In [47]:
df_meta_random

,#SampleID,BarcodeSequence,LinkerPrimerSequence,RunID,Sex,BodySite,Description


In [48]:
df9 = pd.read_csv('data_all.csv')

pbar = tqdm_notebook(range(len(df9))) #to make progressbar
for i in pbar:
    df_meta_random.loc[i] = [df9.loc[i, 'Sample_ID'], \
                            '_', \
                            '_', \
                            df9.loc[i, 'Run_ID'], \
                            df9.loc[i, 'Sex'], \
                            df9.loc[i, 'Body_site'], \
                            '_'
                           ]
df_meta_random

,#SampleID,BarcodeSequence,LinkerPrimerSequence,RunID,Sex,BodySite,Description
0,SRS452123,_,_,SRR921059,female,toeweb,_
1,SRS451751,_,_,SRR920153,female,toeweb,_
2,SRS451761,_,_,SRR920241,female,toeweb,_
3,SRS451782,_,_,SRR920231,female,toeweb,_
4,SRS451473,_,_,SRR919583,female,toeweb,_
5,SRS452059,_,_,SRR920933,male,toeweb,_
6,SRS451727,_,_,SRR920129,male,toeweb,_
7,SRS451935,_,_,SRR920645,male,toeweb,_
8,SRS451454,_,_,SRR919564,male,toeweb,_
9,SRS451618,_,_,SRR919889,male,toeweb,_


In [49]:
df_meta_random.to_csv('07_metadata.txt', sep="\t", index = False)

## Download file lain

In [27]:
cwd = os.getcwd() #get current working directory
os.chdir('D:\Jupyter\KetiakProject\src\sorted files')
import glob
filenames = glob.glob('sort_by_meta*.csv')
df_list = []
amount = 5

for i in range(len(filenames)):
    #print(filenames[i])
    dataframe = pd.read_csv(filenames[i])
    df_random_sample = random_sampling(dataframe, amount)
    df_list.append(df_random_sample)
    random_frame = pd.concat(df_list, ignore_index=False)
    #print('________________________________________________________________')

random_frame
random_frame.to_csv('sorted_metadata.csv', index=False)

os.chdir(cwd)    

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Jupyter\\KetiakProject\\src\\sorted files'

In [163]:
extension = 'JSON Biom'
df_result_m_back = pd.read_csv('ujicoba.csv')

#Ambil data dari EBI
for i in tqdm_notebook(df_result_m_back.index):
    os.chdir(new_dir) #pindah ke folder output
    filename = df_result_m_back.loc[i, "Sample_ID"]+'.biom'
    if not os.path.isfile(filename):
        link = get_analysis_result(df_result_m_back.loc[i, "Run_ID"], extension)
        with open(filename, 'wb') as f:
            c = pycurl.Curl()
            c.setopt(c.URL, link)
            c.setopt(c.WRITEDATA, f)
            c.perform()
            c.close()
    os.chdir(cwd) #balik ke folder semula

NameError: name 'new_dir' is not defined

### References
---
<a id='ref1'></a>
1. Alex L Mitchell, Maxim Scheremetjew, Hubert Denise, Simon Potter, Aleksandra Tarkowska, Matloob Qureshi, Gustavo A Salazar, Sebastien Pesseat, Miguel A Boland, Fiona M I Hunter, Petra ten Hoopen, Blaise Alako, Clara Amid, Darren J Wilkinson, Thomas P Curtis, Guy Cochrane, Robert D Finn; EBI Metagenomics in 2017: enriching the analysis of microbial communities, from sequence reads to assemblies, Nucleic Acids Research, Volume 46, Issue D1, 4 January 2018, Pages D726–D735, https://doi.org/10.1093/nar/gkx967